## ResNet training notebook

In [1]:
from src.args import ResNetTrainingArgs, to_exp_name
from src.data_loading import get_image_data_loader, transforms_image_net
from src.models import MLP, ResNet
from src.test import test_loop
from src.train import training_loop
from src.utils import accuracy, get_optimizer

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

### Params

In [3]:
args = ResNetTrainingArgs(
    batch_size=128,
    dataset_name="mnist",
    learning_rate=0.1,
    num_epochs=200,
    momentum=0.9,
    weight_decay=0.0001,
    cosine_lr=True,
    warmup_epochs=5,
    decay_interval=2,
    decay_gamma=0.1,
    mixup_alpha=0.5,
    mixup_enabled=False,
    lean_stem=True,
    smart_downsampling=True,
    use_gpu=False,
)

dataset_to_n_classes = {
    "mnist": 10,
    "cifar10": 10,
    "fmnist": 10,
}

exp_name = to_exp_name(args)
device = "cuda" if args.use_gpu else "cpu"
if args.use_gpu:
    assert torch.cuda.is_available()
device, exp_name

('cpu', 'mnist_128_0.1_200_0.9_0.0001_True_5_2_0.1_0.5_False_True_True_False')

### Data loading

In [4]:
transform = transforms_image_net(
    crop=True, crop_size=28, flip=True, colors=True, standardize=False,
)

train_data, eval_data = get_image_data_loader(
    args.dataset_name,
    train=True,
    val_share=0.1,
    shuffle=True,
    batch_size=args.batch_size,
    single_batch=False,
    transform=transform,
)

Dataset lengths: train-54000, val-6000


/home/arnaud15/miniconda3/envs/ptorch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630778054/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Model selection

In [5]:
# ImageNet version
# resnet_imagenet = Resnet(
# img_channels=3,
# n_classes=10,
# extra_blocks_per_layer=[1, 3, 5, 2,],
# resnet_channels=[64, 128, 256, 512],
# stem_channels=64,
# stem_downsample=True,
# )
# Cifar10 version
# resnet_cifar = ResNet(
#     img_channels=3,
#     n_classes=10,
#     extra_blocks_per_layer=[5, 5, 5],
#     resnet_channels=[16, 32, 64],
#     stem_channels=16,
#     stem_conv_size=7 if not args.lean_stem else 3,
#     stem_downsample=False,
#     slender_stem=args.lean_stem,
#     better_downsampling=args.smart_downsampling,
# )
# print(summary(resnet_cifar.to(device), (3, 32, 32)))
baby_resnet = ResNet(
    img_channels=1,
    n_classes=10,
    extra_blocks_per_layer=[1, 1, 1],
    resnet_channels=[16, 32, 64],
    stem_channels=16,
    stem_downsample=False,
    slender_stem=False,
    better_downsampling=True,
)
model = baby_resnet
model = model.to(device)

### Training

In [6]:
print(f"Training starts for {exp_name}")
if args.warmup_epochs:
    print(f"Warmup begins")
    no_decay, decay = model.get_params()
    optimizer = get_optimizer(
        decay_params=decay,
        no_decay_params=no_decay,
        lr=args.learning_rate / args.warmup_epochs,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )
    warm_scheduler = optim.lr_scheduler.LambdaLR(
        optimizer, lambda epoch_ix: (epoch_ix + 1)
    )  # 0 indexed epochs
    training_loop(
        name=exp_name,
        model=model,
        opt=optimizer,
        scheduler=warm_scheduler,
        train_loader=train_data,
        eval_loader=eval_data,
        loss_fn=F.cross_entropy,
        metric_fn=accuracy,
        n_epochs=args.warmup_epochs,
        device=device,
        plot_every=5,
        print_every=1,
        write_every=1,
        check_every=0,
    )
no_decay, decay = model.get_params()
optimizer = get_optimizer(
    decay_params=decay,
    no_decay_params=no_decay,
    lr=args.learning_rate,
    momentum=args.momentum,
    weight_decay=args.weight_decay,
)
if not args.cosine_lr:
    base_scheduler = optim.lr_scheduler.StepLR(
        optimizer, step_size=args.decay_interval, gamma=args.decay_gamma
    )
else:
    base_scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, args.num_epochs, 0
    )
print(f"Post-warmup begins")

training_loop(
    name=exp_name,
    model=model,
    opt=optimizer,
    scheduler=base_scheduler,
    train_loader=train_data,
    eval_loader=eval_data,
    loss_fn=F.cross_entropy,
    metric_fn=accuracy,
    n_epochs=args.num_epochs,
    device=device,
    print_every=1000,
    write_every=100,
    check_every=10000,
)

Training starts for mnist_128_0.1_200_0.9_0.0001_True_5_2_0.1_0.5_False_True_True_False
Warmup begins
Start of epoch 1
Step: 1 | Training Loss: 2.41721
Step: 1 | Training Metric: 0.09375
Step: 2 | Training Loss: 2.36898
Step: 2 | Training Metric: 0.07812
Step: 3 | Training Loss: 2.33173
Step: 3 | Training Metric: 0.13281
Step: 4 | Training Loss: 2.28548
Step: 4 | Training Metric: 0.15625
Step: 5 | Training Loss: 2.16987
Step: 5 | Training Metric: 0.31250
Step: 6 | Training Loss: 2.21204
Step: 6 | Training Metric: 0.23438


KeyboardInterrupt: 

### Load from checkpoint

In [ ]:
loaded = torch.load(
    os.path.join(os.path.join("data", "checkpoints"), f"{exp_name}-70000.pt"),
    map_location=device,
)
model.load_state_dict(loaded["model_state"])

### Load test data

In [ ]:
test_loader, should_be_none = get_image_data_loader(
    args.dataset_name,
    train=False,
    val_share=0.1,
    shuffle=True,
    batch_size=args.batch_size,
    single_batch=False,
)
assert should_be_none is None

### Evaluate the loaded model

In [ ]:
test_loop(
    test_loader=test_loader,
    model=model,
    device=device,
    metric_fn=accuracy,
    plot=True,
    loss_fn=F.cross_entropy,
)